In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import sys
from importlib import reload
import cv2
import json

print('Python | Pytorch | OpenCV versions: %s | %s | %s' %(sys.version, torch.__version__, cv2.__version__))

sys.path.append('../../')
import tools
from tools.utils import *
from tools.visuals import *
# import model
# from model.model import * 
# from train_eval import *
# from test import *

if __name__ == "__main__":
    print("lol")

Python | Pytorch | OpenCV versions: 3.7.0 (default, Jun 28 2018, 13:15:42) 
[GCC 7.2.0] | 0.4.1 | 3.4.3
lol


In [3]:
import os, sys
import xmltodict
from pathlib import Path
import numpy as np
from skimage import io
import operator
from functools import reduce


def xy_xy2ulxy_wh(x):  # Convert bounding box format from [x1, y1, x2, y2] to [x, y, w, h]
    y = np.zeros(len(x))
    y[0] = x[0]
    y[1] = x[1]
    y[2] = x[2] - x[0]
    y[3] = x[3] - x[1]
    return y.tolist()

def xy_xy2bbxy_wh(x):  # Convert bounding box format from [x1, y1, x2, y2] to [x, y, w, h]
    y = np.zeros(len(x))
    y[0] = (x[0] + x[2]) / 2
    y[1] = (x[1] + x[3]) / 2
    y[2] = x[2] - x[0]
    y[3] = x[3] - x[1]
    return y.tolist()

# def adjust_polygon_format(o_polygon):
#     polygon_list = []
#     co_k_x = sorted(list(o_polygon.keys()), key=lambda x: int(x[1:]))
#     for i in co_k_x:
#         ix = i[1:]
#         xk, yk = i, 'y' + ix
#         polygon_list.append([float(o_polygon[xk]), float(o_polygon[yk])])

#     polygon_list = reduce(operator.add, polygon_list)    
#     return polygon_list
        
def adjust_polygon_format(o_polygon):
    polygon_list = []
    co_k_x = [i for i in list(o_polygon.keys()) if i[0] == 'x']
    co_k_x = sorted(co_k_x, key=lambda x: int(x[1:]))
    for i in co_k_x:
        ix = i[1:]
        xk, yk = i, 'y' + ix
        polygon_list.append([float(o_polygon[xk]), float(o_polygon[yk])])
        
    polygon_list = reduce(operator.add, polygon_list)
    
    return polygon_list

# from shapely.geometry import Polygon

# def polygon_area(coords):
#     polygon = Polygon(coords)
#     print(polygon.area)

In [4]:
# from collections import OrderedDict
# def adjust_polygon_format(o_polygon):
#     polygon_list = []
#     co_k_x = [i for i in list(o_polygon.keys()) if i[0] == 'x']
#     co_k_x = sorted(co_k_x, key=lambda x: int(x[1:]))
#     print(co_k_x)
#     for i in co_k_x:
#         ix = i[1:]
#         xk, yk = i, 'y' + ix
#         polygon_list.append([float(o_polygon[xk]), float(o_polygon[yk])])
        
#     print(polygon_list)
#     polygon_list = reduce(operator.add, polygon_list)
#     print(polygon_list)
    
#     return polygon_list

# o_polygon = OrderedDict([('y13', '658'), ('y25', '575'), ('y18', '624'), ('y21', '599'), ('y10', '622'), ('y7', '599'), ('y11', '626'), ('y8', '613'), ('x26', '1313'), ('y19', '614'), ('y5', '588'), ('y28', '561'), ('x16', '1262'), ('y22', '593'), ('x9', '1369'), ('x12', '1359'), ('y14', '660'), ('x19', '1253'), ('y20', '606'), ('x10', '1367'), ('y17', '634'), ('x29', '1341'), ('x15', '1278'), ('y16', '648'), ('x7', '1375'), ('x3', '1358'), ('y24', '582'), ('y1', '561'), ('x4', '1360'), ('y6', '594'), ('y26', '568'), ('y4', '582'), ('y3', '574'), ('y2', '562'), ('x14', '1290'), ('x6', '1368'), ('x20', '1257'), ('y23', '587'), ('x28', '1326'), ('x22', '1269'), ('y9', '618'), ('x27', '1319'), ('y15', '658'), ('y12', '628'), ('x13', '1301'), ('x24', '1288'), ('x5', '1363'), ('x18', '1252'), ('x1', '1351'), ('y29', '558'), ('x2', '1356'), ('x8', '1371'), ('x11', '1363'), ('y27', '564'), ('x25', '1301'), ('x23', '1279'), ('x21', '1262'), ('x17', '1254')])
# adjust_polygon_format(o_polygon)

In [4]:
with open(str(cat_file), 'r') as f:
    cat_data = [i.strip() for i in f.readlines()]

# cat_dic = {cat_file[i]:i for i in range(len(cat_file))}
cat_dic = {i:cat_data[i] for i in range(len(cat_data))}
# print(cat_dic)
    
### CATEGORIES ###

categories = [{
    "id" : i, #int, 
    "name" : j, #str, 
    "supercategory" : j, #str,
} for i, j in cat_dic.items()]
    
### DATASET ###

info = {
    "year" : '2018', 
    "version" : '1', 
    "description" : 'fumehood', 
    "contributor" : 'v7', 
    "url" : 'none', 
    "date_created" : '2018/09/20',
}

dataset = {
    
    "info" : info, 
    "images" : [],       # image
    "annotations" : [],  # annotation
    "licenses" : [],     # license
    "categories": categories
}

count_i = 0
count_o = 0
for file in ann_files:
    with open(str(file)) as fd:
        doc = xmltodict.parse(fd.read())
        
        count_i += 1
        # creating an image: 
        data = doc['annotation']
        # image
        folder = data['folder']
        filename = data['filename']
        url = folder + '/' + filename
        im_name = str(file).split('/')[-1].split('.')[0] + '.png'    # filename.split('.')[0]
        width = int(data['size']['width'])
        height = int(data['size']['height'])
        objects = data['object']
        
        image_dic = {
            "id" : count_i, #int, 
            "width" : width, #int, 
            "height" : height, #int, 
            "file_name" : im_name, #str, 
            "license" : 0, #int, 
            "flickr_url" : 'no-flickr_url', #str, 
            "coco_url" : url, #str, 
            "date_captured" : "2018-20-01 00:00:00", #datetime,
        }
        
        license = {
            "id" : count_i, #int, 
            "name" : im_name, #str, 
            "url" : 'url', #str,
        }
        
        dataset['images'].append(image_dic)
        dataset['licenses'].append(image_dic)
        
        # parsing objects annotations
        if isinstance(objects, dict): objects = [objects]
        
#         count_o = 0
        for i in objects:
            count_o += 1
            
            o_cat_name = i['name']
            o_cat_ix = cat_data.index(o_cat_name)
            o_polygon = i['polygon']
            o_polygon = [adjust_polygon_format(o_polygon)]
            o_bbox = [i['bndbox']['xmin'], i['bndbox']['ymin'], i['bndbox']['xmax'], i['bndbox']['ymax']]
            o_bbox = xy_xy2ulxy_wh([int(i) for i in o_bbox])
            
            annotation = {
                "id" : count_o, #int, 
                "image_id" : count_i, #int, 
                "category_id" : o_cat_ix, #int, 
                "segmentation" : o_polygon, # polygon format or list 
                "area" : 33., #float, 
                "bbox" : o_bbox, #[x, y, width, height], 
                "iscrowd" : 0, # 0 or 1,
            }
            
            dataset['annotations'].append(annotation)

# print(json.dumps(dataset, indent=1))

#### COCO annot. example

In [89]:
# coco data

path = '/home/blanca/mnt/projects/detector/data/annotations'
# ctype = 'captions_train2017.json'
ctype = 'instances_val2017.json'

file = path + '/' + ctype
with open(file) as f:
    cdata = json.load(f)

# # jdata overview
# for i in jdata.keys(): print(i)

# for i in jdata.keys():
#     print(jdata['categories'])
    
print(json.dumps(jdata, indent=2))

of = 'coco_data_sample.txt'
with open(of, 'w') as f:
    json.dump(cdata, f, indent=1)

In [112]:
# 'info':
{'description': 'COCO 2017 Dataset',
  'url': 'http://cocodataset.org',
  'version': '1.0',
  'year': 2017,
  'contributor': 'COCO Consortium',
  'date_created': '2017/09/01'}

{'description': 'COCO 2017 Dataset',
 'url': 'http://cocodataset.org',
 'version': '1.0',
 'year': 2017,
 'contributor': 'COCO Consortium',
 'date_created': '2017/09/01'}

In [113]:
# 'image':
{"license": 3,"file_name": "000000512403.jpg","coco_url": "http://images.cocodataset.org/val2017/000000512403.jpg","height": 640,"width": 529,"date_captured": "2013-11-24 05:12:53","flickr_url": "http://farm1.staticflickr.com/143/350452845_fa743a9623_z.jpg","id": 512403}

{'license': 3,
 'file_name': '000000512403.jpg',
 'coco_url': 'http://images.cocodataset.org/val2017/000000512403.jpg',
 'height': 640,
 'width': 529,
 'date_captured': '2013-11-24 05:12:53',
 'flickr_url': 'http://farm1.staticflickr.com/143/350452845_fa743a9623_z.jpg',
 'id': 512403}

In [114]:
# 'annotation':
{"segmentation": [[510.66,423.01,511.72,420.03,510.45,416.0,510.34,413.02,510.77,410.26,510.77,407.5,510.34,405.16,511.51,402.83,511.41,400.49,510.24,398.16,509.39,397.31,504.61,399.22,502.17,399.64,500.89,401.66,500.47,402.08,499.09,401.87,495.79,401.98,490.59,401.77,488.79,401.77,485.39,398.58,483.9,397.31,481.56,396.35,478.48,395.93,476.68,396.03,475.4,396.77,473.92,398.79,473.28,399.96,473.49,401.87,474.56,403.47,473.07,405.59,473.39,407.71,476.68,409.41,479.23,409.73,481.56,410.69,480.4,411.85,481.35,414.93,479.86,418.65,477.32,420.03,476.04,422.58,479.02,422.58,480.29,423.01,483.79,419.93,486.66,416.21,490.06,415.57,492.18,416.85,491.65,420.24,492.82,422.9,493.56,424.39,496.43,424.6,498.02,423.01,498.13,421.31,497.07,420.03,497.07,415.15,496.33,414.51,501.1,411.96,502.06,411.32,503.02,415.04,503.33,418.12,501.1,420.24,498.98,421.63,500.47,424.39,505.03,423.32,506.2,421.31,507.69,419.5,506.31,423.32,510.03,423.01,510.45,423.01]],"area": 702.1057499999998,"iscrowd": 0,"image_id": 289343,"bbox": [473.07,395.93,38.65,28.67],"category_id": 18,"id": 1768}

{'segmentation': [[510.66,
   423.01,
   511.72,
   420.03,
   510.45,
   416.0,
   510.34,
   413.02,
   510.77,
   410.26,
   510.77,
   407.5,
   510.34,
   405.16,
   511.51,
   402.83,
   511.41,
   400.49,
   510.24,
   398.16,
   509.39,
   397.31,
   504.61,
   399.22,
   502.17,
   399.64,
   500.89,
   401.66,
   500.47,
   402.08,
   499.09,
   401.87,
   495.79,
   401.98,
   490.59,
   401.77,
   488.79,
   401.77,
   485.39,
   398.58,
   483.9,
   397.31,
   481.56,
   396.35,
   478.48,
   395.93,
   476.68,
   396.03,
   475.4,
   396.77,
   473.92,
   398.79,
   473.28,
   399.96,
   473.49,
   401.87,
   474.56,
   403.47,
   473.07,
   405.59,
   473.39,
   407.71,
   476.68,
   409.41,
   479.23,
   409.73,
   481.56,
   410.69,
   480.4,
   411.85,
   481.35,
   414.93,
   479.86,
   418.65,
   477.32,
   420.03,
   476.04,
   422.58,
   479.02,
   422.58,
   480.29,
   423.01,
   483.79,
   419.93,
   486.66,
   416.21,
   490.06,
   415.57,
   492.18,
   416.85,

#### convert fumehood 

In [5]:
xml_annotations_path = Path('/home/datasets/fh/Annotations')
path = Path(xml_annotations_path)
ann_files = list(path.glob("*")) 
cat_file = path.parent / Path('fumehood.names') # categories names file path
cat_file

PosixPath('/home/datasets/fh/fumehood.names')

In [12]:
pT = '/home/datasets/fh/JPEGImagesTrain/'
pV = '/home/datasets/fh/JPEGImagesVal/'
lT = [i.name for i in list(Path(pT).glob('*.png'))]
lV = [i.name for i in list(Path(pV).glob('*.png'))]

print(len(lT))
print(len(lV))

with open(str(cat_file), 'r') as f:
    cat_data = [i.strip() for i in f.readlines()]

cat_dic = {i:cat_data[i] for i in range(len(cat_data))}
print(cat_dic)
    
### CATEGORIES ###

categories = [{
    "id" : i, #int, 
    "name" : j, #str, 
    "supercategory" : j, #str,
} for i, j in cat_dic.items()]

### DATASET ###

info = {
    "year" : '2018', 
    "version" : '1', 
    "description" : 'fumehood', 
    "contributor" : 'v7', 
    "url" : 'none', 
    "date_created" : '2018/09/20',
}

859
48
{0: 'beaker', 1: 'bottle', 2: 'flask', 3: 'bare_hand', 4: 'glove_hand', 5: 'pipette', 6: 'pipette_label', 7: 'bottle_cap'}


In [15]:
cat_data

['beaker',
 'bottle',
 'flask',
 'bare_hand',
 'glove_hand',
 'pipette',
 'pipette_label',
 'bottle_cap']

In [14]:
train_dataset = {
    
    "info" : info, 
    "images" : [],       # image
    "annotations" : [],  # annotation
    "licenses" : [],     # license
    "categories": categories
}

val_dataset = {
    
    "info" : info, 
    "images" : [],       # image
    "annotations" : [],  # annotation
    "licenses" : [],     # license
    "categories": categories
}

# populate datasets train/val

count_i = 0
count_o = 0
for file in ann_files:
    with open(str(file)) as fd:
        doc = xmltodict.parse(fd.read())
        
        count_i += 1
        # creating an image: 
        data = doc['annotation']
        # image
        folder = data['folder']
        filename = data['filename']
        url = folder + '/' + filename
        im_name = str(file).split('/')[-1].split('.')[0] + '.png'    # filename.split('.')[0]
        width = int(data['size']['width'])
        height = int(data['size']['height'])
        objects = data['object']
        
        image = {
            "id" : count_i, #int, 
            "width" : width, #int, 
            "height" : height, #int, 
            "file_name" : im_name, #str, 
            "license" : 0, #int, 
            "flickr_url" : 'no-flickr_url', #str, 
            "coco_url" : url, #str, 
            "date_captured" : "2018-20-01 00:00:00", #datetime,
        }
        
        license = {
            "id" : count_i, #int, 
            "name" : im_name, #str, 
            "url" : 'url', #str,
        }
        
        if im_name in lT: 
            train_dataset['images'].append(image)
            train_dataset['licenses'].append(license)
        elif im_name in lV: 
            val_dataset['annotations'].append(image)
            val_dataset['licenses'].append(license)
        else: print('Ohpsss')
        
        # parsing objects annotations
        if isinstance(objects, dict): objects = [objects]
        
        for i in objects:
            count_o += 1
            
            o_cat_name = i['name']
            o_cat_ix = cat_data.index(o_cat_name)
            o_polygon = i['polygon']
            o_polygon = [adjust_polygon_format(o_polygon)]
            o_bbox = [i['bndbox']['xmin'], i['bndbox']['ymin'], i['bndbox']['xmax'], i['bndbox']['ymax']]
            o_bbox = xy_xy2ulxy_wh([int(i) for i in o_bbox])
            
            annotation = {
                "id" : count_o, #int, 
                "image_id" : count_i, #int, 
                "category_id" : o_cat_ix, #int, 
                "segmentation" : o_polygon, # polygon format or list 
                "area" : 33., #float, 
                "bbox" : o_bbox, #[x, y, width, height], 
                "iscrowd" : 0, # 0 or 1,
            }

            if im_name in lT: train_dataset['annotations'].append(annotation)
            elif im_name in lV: val_dataset['annotations'].append(annotation)
            else: print('Ohpsss')

print(len(train_dataset['images']), len(train_dataset['licenses']), len(train_dataset['annotations']), \
      len(val_dataset['images']), len(val_dataset['licenses']), len(val_dataset['annotations']))

355 355 1489 0 35 158


In [21]:
# save as .json
out_file_train = '/home/datasets/fh/annotations/annotations_train.json'
out_file_train = '/home/datasets/fh/annotations/annotations_val.json'

with open(out_file_train, 'w') as f:
    json.dump(train_dataset, f, indent=1)

with open(out_file_val, 'w') as f:
    json.dump(val_dataset, f, indent=1)

In [22]:
# class NumpyEncoder(json.JSONEncoder):
#     """ Special json encoder for numpy types """
#     def default(self, obj):
#         if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
#             np.int16, np.int32, np.int64, np.uint8,
#             np.uint16, np.uint32, np.uint64)):
#             return int(obj)
#         elif isinstance(obj, (np.float_, np.float16, np.float32, 
#             np.float64)):
#             return float(obj)
#         elif isinstance(obj,(np.ndarray,)): #### This is the fix
#             return obj.tolist()
#         return json.JSONEncoder.default(self, obj)

# save as .json
out_file = '/home/datasets/fh/annotations/annotations.json'
# out_file = '/home/blanca/mnt/fh/annotations/annotations.json'

with open(out_file, 'w') as f:
    json.dump(dataset, f, indent=1)

# # to print
# of = 'fh_data_sample.txt'
# with open(of, 'w') as f:
#     json.dump(dataset, f, indent=1)

In [3]:

# 3_000061_0

X-101-64x4d-FPN

X-152-32x8d-FPN-IN5k


def download_images_and_parse_labels(annotations_path, save_images=False): 
    files, nO_files = 0, 0
    path = Path(annotations_path)
    ann_files = list(path.glob("*")) 
    cat_file = path.parent / Path('fumehood.names') # categories names file path
    
    with open(str(cat_file), 'r') as f:
        cat_file = [i.strip() for i in f.readlines()]
    
    dataset = {} # store all dataset labels in a dic (for now) - format {m_name: {image: np, label: list}
    for file in ann_files:
        with open(str(file)) as fd:
            doc = xmltodict.parse(fd.read())

        data = doc['annotation']
        folder = data['folder']
        try : objects = data['object']
        except: 
            nO_files += 1
            print('No object labeled in file %s' %file)
            continue
        filename = data['filename']
        session = folder.split('/')[-1]
        im_name = filename.split('.')[0] + '_' + session
        
        # retrieve the image
        url = folder + '/' + filename
        dpath = '../data/fumehood/images/'
        im = io.imread(url)
        if save_images: np.save('%s%s.npy'%(dpath, im_name), im)
        
        # retrieve and parse the label (each 'label' - multiple annotations: one image multiple labels)
        label = []
        if isinstance(objects, dict): objects = [objects]
        for i in objects:
            cat_name = i['name']
            polygon = i['polygon']
            bbox = (i['bndbox']['xmin'], i['bndbox']['xmax'], i['bndbox']['ymin'], i['bndbox']['ymax'])
            cat = [cat_file.index(cat_name)]
            lab = np.array(np.concatenate([cat, bbox]), dtype='float32')
            label.append(lab)
        
        files += 1
        if im_name in dataset: print(dataset[im_name]['file'], file)
        dataset[im_name] = {'image': im, 'label': label, 'file': file}
    # save the labels in txt files and read from them?
    return dataset, files, nO_files, len(ann_files)

path = '../data/fumehood/annotations'
dataset, files, nO_files, _ = download_images_and_parse_labels(path)
nO_files, _

In [44]:
## DEBUG

    clss = bbox_target_data[:, 0]
    bbox_targets = blob_utils.zeros((clss.size, 4 * num_bbox_reg_classes))
    bbox_inside_weights = blob_utils.zeros(bbox_targets.shape)
    inds = np.where(clss > 0)[0]
    for ind in inds:
        cls = int(clss[ind])
        start = 4 * cls
        end = start + 4
        bbox_targets[ind, start:end] = bbox_target_data[ind, 1:]
        bbox_inside_weights[ind, start:end] = (1.0, 1.0, 1.0, 1.0)
    return bbox_targets, bbox_inside_weights

In [50]:
s = np.array(
    [[ 8.,          0.38198203, -1.3716389,  -0.50592613, -0.9699857],
     [ 8.,         -0.5990037,   1.143877,   -0.83646095,  1.2254364 ]])
s

array([[ 8.        ,  0.38198203, -1.3716389 , -0.50592613, -0.9699857 ],
       [ 8.        , -0.5990037 ,  1.143877  , -0.83646095,  1.2254364 ]])

In [59]:
def zeros(shape, int32=False):
    """Return a blob of all zeros of the given shape with the correct float or
    int data type.
    """
    return np.zeros(shape, dtype=np.int32 if int32 else np.float32)

bbox_target_data = np.zeros((256, 5))


num_bbox_reg_classes = 8
clss = bbox_target_data[:, 0]
print(clss.size)

bbox_targets = zeros((clss.size, 4 * num_bbox_reg_classes))
print(bbox_targets.shape)
bbox_inside_weights = zeros(bbox_targets.shape)
inds = np.where(clss > 0)[0]
for ind in inds:
    cls = int(clss[ind])
    start = 4 * cls
    end = start + 4
    bbox_targets[ind, start:end] = bbox_target_data[ind, 1:]
    bbox_inside_weights[ind, start:end] = (1.0, 1.0, 1.0, 1.0)
    

# return bbox_targets, bbox_inside_weights
bbox_inside_weights

256
(256, 32)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [76]:
detections_path = '/tmp/detectron-output/test/fumehood_train/generalized_rcnn'
detection_file = 'detections.pkl'

import pickle
data = pickle.load(open(detection_file,'rb'), encoding='latin1') #, errors='ignore'
for i, j in data.items():
    print(i)
data['all_boxes']

cfg
all_segms
all_keyps
all_boxes


[[],
 [array([], shape=(0, 5), dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([[9.4227344e+02, 6.0956403e+02, 1.0875532e+03, 8.1106635e+02,
          9.9975330e-01]], dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([[9.4303223e+02, 6.0856714e+02, 1.0875232e+03, 8.1111658e+02,
          9.9997270e-01]], dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([[9.4161761e+02, 6.0854144e+02, 1.0884291e+03, 8.1342462e+02,
          9.9995792e-01]], dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([], shape=(0, 5), 

In [43]:
pT = '/home/datasets/fh/JPEGImagesTrain/'
pV = '/home/datasets/fh/JPEGImagesVal/'
lT = [i.name for i in list(Path(pT).glob('*.png'))]
lV = [i.name for i in list(Path(pV).glob('*.png'))]

print(len(lT))
print(len(lV))

861
50


In [41]:
p = '/home/blanca/mnt/fh/'
p1 = 'JPEGImagesTest/'
p2 = 'JPEGImagesTrain/'

# on = list(Path(p + p2).glob('*.png'))
# print(len(on))
# delete = [i.name for i in list(Path(p + p1).glob('*.png'))]
# print(len(delete))
# for i in delete:
#     os.remove(p + p2 + i)
on = list(Path(p + p2).glob('*.png'))
print(len(on))
delete = [i.name for i in list(Path(p + p1).glob('*.png'))]
print(len(delete))

861
50


# Test on Recordings

In [48]:
p = '/home/blanca/recordings/'
folders = ['Images0', 'Images1', 'Images2']
[os.makedirs(p + f, exist_ok=True) for f in folders]

[None, None, None]

In [54]:
for f in folders:
    out_p = p + f
    file = p + f[-1]+'.mp4'
    vidcap = cv2.VideoCapture(file)
    success, image = vidcap.read()
    count = 0
    while success:
        count += 1
        width = 10
        im_name = str(count).zfill(width) + '.jpg'
        cv2.imwrite(out_p + '/' + im_name, image)     # save frame as JPEG file      
        success, image = vidcap.read()
        print('Wrote a new frame as: ', out_p + '/' + im_name)
# 449, 449, 450

Wrote a new frame as:  /home/blanca/recordings/Images0/0000000001.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000002.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000003.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000004.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000005.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000006.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000007.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000008.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000009.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000010.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000011.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000012.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000013.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000014.jpg
Wrote a new frame as

Wrote a new frame as:  /home/blanca/recordings/Images0/0000000127.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000128.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000129.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000130.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000131.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000132.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000133.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000134.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000135.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000136.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000137.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000138.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000139.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000140.jpg
Wrote a new frame as

Wrote a new frame as:  /home/blanca/recordings/Images0/0000000247.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000248.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000249.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000250.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000251.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000252.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000253.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000254.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000255.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000256.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000257.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000258.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000259.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000260.jpg
Wrote a new frame as

Wrote a new frame as:  /home/blanca/recordings/Images0/0000000369.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000370.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000371.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000372.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000373.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000374.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000375.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000376.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000377.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000378.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000379.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000380.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000381.jpg
Wrote a new frame as:  /home/blanca/recordings/Images0/0000000382.jpg
Wrote a new frame as

Wrote a new frame as:  /home/blanca/recordings/Images1/0000000041.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000042.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000043.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000044.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000045.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000046.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000047.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000048.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000049.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000050.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000051.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000052.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000053.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000054.jpg
Wrote a new frame as

...

Wrote a new frame as:  /home/blanca/recordings/Images1/0000000398.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000399.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000400.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000401.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000402.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000403.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000404.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000405.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000406.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000407.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000408.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000409.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000410.jpg
Wrote a new frame as:  /home/blanca/recordings/Images1/0000000411.jpg
Wrote a new frame as

Wrote a new frame as:  /home/blanca/recordings/Images2/0000000072.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000073.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000074.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000075.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000076.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000077.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000078.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000079.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000080.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000081.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000082.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000083.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000084.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000085.jpg
Wrote a new frame as

Wrote a new frame as:  /home/blanca/recordings/Images2/0000000194.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000195.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000196.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000197.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000198.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000199.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000200.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000201.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000202.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000203.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000204.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000205.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000206.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000207.jpg
Wrote a new frame as

Wrote a new frame as:  /home/blanca/recordings/Images2/0000000315.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000316.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000317.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000318.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000319.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000320.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000321.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000322.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000323.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000324.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000325.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000326.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000327.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000328.jpg
Wrote a new frame as

Wrote a new frame as:  /home/blanca/recordings/Images2/0000000438.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000439.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000440.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000441.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000442.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000443.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000444.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000445.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000446.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000447.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000448.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000449.jpg
Wrote a new frame as:  /home/blanca/recordings/Images2/0000000450.jpg


## Create Video

In [77]:
import cv2
import os

suffix = '2_c_0.9'

input_dir = '.'
output_dir = '.'

input_dir = '/home/blanca/mnt/tmp/r%s/' %suffix
output_dir = '/home/blanca/mnt/Detectron_VisOutput/'
output_name = output_dir + 'recordings-%s-e5000-b256.avi' %suffix

images = sorted([img for img in np.sort(os.listdir(input_dir)) if img.endswith(".jpg")])

frame = cv2.imread(os.path.join(input_dir, images[0]))
height, width, layers = frame.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(output_name, fourcc, 15.0, (width, height))

for image in images:
    video.write(cv2.imread(os.path.join(input_dir, image)))

cv2.destroyAllWindows()
video.release()


#### colormap

In [55]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy as np


def colormap(rgb=False):
    color_list = np.array(
        [
            0.000, 0.447, 0.741,
            0.850, 0.325, 0.098,
            0.929, 0.694, 0.125,
            0.494, 0.184, 0.556,
            0.466, 0.674, 0.188,
            0.301, 0.745, 0.933,
            0.635, 0.078, 0.184,
            0.300, 0.300, 0.300,
            0.600, 0.600, 0.600,
            1.000, 0.000, 0.000,
            1.000, 0.500, 0.000,
            0.749, 0.749, 0.000,
            0.000, 1.000, 0.000,
            0.000, 0.000, 1.000,
            0.667, 0.000, 1.000,
            0.333, 0.333, 0.000,
            0.333, 0.667, 0.000,
            0.333, 1.000, 0.000,
            0.667, 0.333, 0.000,
            0.667, 0.667, 0.000,
            0.667, 1.000, 0.000,
            1.000, 0.333, 0.000,
            1.000, 0.667, 0.000,
            1.000, 1.000, 0.000,
            0.000, 0.333, 0.500,
            0.000, 0.667, 0.500,
            0.000, 1.000, 0.500,
            0.333, 0.000, 0.500,
            0.333, 0.333, 0.500,
            0.333, 0.667, 0.500,
            0.333, 1.000, 0.500,
            0.667, 0.000, 0.500,
            0.667, 0.333, 0.500,
            0.667, 0.667, 0.500,
            0.667, 1.000, 0.500,
            1.000, 0.000, 0.500,
            1.000, 0.333, 0.500,
            1.000, 0.667, 0.500,
            1.000, 1.000, 0.500,
            0.000, 0.333, 1.000,
            0.000, 0.667, 1.000,
            0.000, 1.000, 1.000,
            0.333, 0.000, 1.000,
            0.333, 0.333, 1.000,
            0.333, 0.667, 1.000,
            0.333, 1.000, 1.000,
            0.667, 0.000, 1.000,
            0.667, 0.333, 1.000,
            0.667, 0.667, 1.000,
            0.667, 1.000, 1.000,
            1.000, 0.000, 1.000,
            1.000, 0.333, 1.000,
            1.000, 0.667, 1.000,
            0.167, 0.000, 0.000,
            0.333, 0.000, 0.000,
            0.500, 0.000, 0.000,
            0.667, 0.000, 0.000,
            0.833, 0.000, 0.000,
            1.000, 0.000, 0.000,
            0.000, 0.167, 0.000,
            0.000, 0.333, 0.000,
            0.000, 0.500, 0.000,
            0.000, 0.667, 0.000,
            0.000, 0.833, 0.000,
            0.000, 1.000, 0.000,
            0.000, 0.000, 0.167,
            0.000, 0.000, 0.333,
            0.000, 0.000, 0.500,
            0.000, 0.000, 0.667,
            0.000, 0.000, 0.833,
            0.000, 0.000, 1.000,
            0.000, 0.000, 0.000,
            0.143, 0.143, 0.143,
            0.286, 0.286, 0.286,
            0.429, 0.429, 0.429,
            0.571, 0.571, 0.571,
            0.714, 0.714, 0.714,
            0.857, 0.857, 0.857,
            1.000, 1.000, 1.000
        ]
    ).astype(np.float32)
    color_list = color_list.reshape((-1, 3)) * 255
    if not rgb:
        color_list = color_list[:, ::-1]
    return color_list

In [56]:
color_list = colormap(rgb=True) / 255


In [58]:
mask_color_id = 0
for i in range (20):
    color_mask = color_list[mask_color_id % len(color_list), 0:3]
    mask_color_id += 1
    print(color_mask)

[0.    0.447 0.741]
[0.85  0.325 0.098]
[0.929 0.694 0.125]
[0.494 0.184 0.556]
[0.466 0.674 0.188]
[0.301 0.745 0.933]
[0.635 0.078 0.184]
[0.3 0.3 0.3]
[0.6 0.6 0.6]
[1. 0. 0.]
[1.  0.5 0. ]
[0.749 0.749 0.   ]
[0. 1. 0.]
[0. 0. 1.]
[0.667 0.    1.   ]
[0.333 0.333 0.   ]
[0.333 0.667 0.   ]
[0.333 1.    0.   ]
[0.667 0.333 0.   ]
[0.667 0.667 0.   ]


In [60]:
classes = cat_dic
classes

{0: 'beaker',
 1: 'bottle',
 2: 'flask',
 3: 'bare_hand',
 4: 'glove_hand',
 5: 'pipette',
 6: 'pipette_label',
 7: 'bottle_cap'}

In [62]:
color_dict = {
    '__background__': [0.   , 0.447, 0.741],
    'beaker':         [0.85 , 0.325, 0.098],
    'bottle':         [0.929, 0.694, 0.125],
    'flask':          [0.494, 0.184, 0.556],
    'bare_hand':      [0.466, 0.674, 0.188],
    'glove_hand':     [0.301, 0.745, 0.933],
    'pipette':        [0.635, 0.078, 0.184],
    'pipette_label':  [0.3  , 0.3  , 0.3  ],
    'bottle_cap':     [0.6  , 0.6  , 0.6  ],
}

In [65]:
mask_color_id = 0
for i in range (9):
    color_mask = color_list[mask_color_id % len(color_list), 0:3]
    mask_color_id += 1
    color_mask_fixed = color_dict[i]
    print(color_mask, color_mask_fixed)

[0.    0.447 0.741] [0.0, 0.447, 0.741]
[0.85  0.325 0.098] [0.85, 0.325, 0.098]
[0.929 0.694 0.125] [0.929, 0.694, 0.125]
[0.494 0.184 0.556] [0.494, 0.184, 0.556]
[0.466 0.674 0.188] [0.466, 0.674, 0.188]
[0.301 0.745 0.933] [0.301, 0.745, 0.933]
[0.635 0.078 0.184] [0.635, 0.078, 0.184]
[0.3 0.3 0.3] [0.3, 0.3, 0.3]
[0.6 0.6 0.6] [0.6, 0.6, 0.6]
